In [1]:
import numpy as np
import pandas as pd
import math
import warnings
import datetime

warnings.simplefilter("ignore")

In [2]:
case = pd.read_csv("data/data/covid_19.csv")

In [3]:
case["submission_date"] = case["submission_date"].astype("datetime64")
case = case.loc[:, ["submission_date", "state", "tot_cases", "new_case"]]
case["lag_date"] = case["submission_date"].shift(1)

In [4]:
join = pd.merge(case, case, how="left", left_on = ["submission_date", "state"], right_on = ["lag_date", "state"])
join["inc_rate"] = join["new_case_y"] / join["tot_cases_x"] 
join = join.fillna(0)

In [5]:
def helper1(state):
    tmp = join[join["state"] == state].reset_index(drop=True)
    inf_ind = np.where(tmp.inc_rate==math.inf)[0]
    return tmp.loc[inf_ind[0]+1:, ["submission_date_x", "state", "inc_rate"]] if inf_ind else tmp.loc[:, ["submission_date_x", "state", "inc_rate"]]
    
case_ = pd.concat([helper1(x) for x in np.unique(join["state"])],axis=0).reset_index(drop=True)
case_.columns = ["date", "state", "inc_rate"]

In [6]:
policy = pd.read_csv("data/data/policy.csv")
policy = policy.loc[:, ['POSTCODE', 'STAYHOME(start)', 'FM_ALL(start)', 'CLREST(start)', 'CLBSNS(start)']]
for v in ['STAYHOME(start)', 'FM_ALL(start)', 'CLREST(start)', 'CLBSNS(start)']:
    policy[v] = policy[v].replace("0", None)
    policy = policy.dropna()
    policy[v] = policy[v].astype('datetime64')
policy.columns = ['state', 'stay at home order', 'face mask mandatory', 'restaurant close', 'business close']
policy = pd.melt(policy, id_vars="state", var_name="policy", value_name="date")

policy["before_7"] = policy["date"]+datetime.timedelta(days=-7)
policy["after_7"] = policy["date"]+datetime.timedelta(days=7)
policy["before_15"] = policy["date"]+datetime.timedelta(days=-15)
policy["after_15"] = policy["date"]+datetime.timedelta(days=15)

In [59]:
res = []

for s in policy.state.unique():
    for p in policy.policy.unique():
        tmp = policy.loc[np.logical_and(policy["state"] == s, policy["policy"] == p),:]
        date = tmp['date'][tmp.index[0]]
        tmp_ = case_.loc[case_["state"] == s,:]
        tmp_b7 = tmp_.loc[np.logical_and(case_["date"] >= tmp['before_7'][tmp.index[0]],
                                     case_["date"] < tmp['date'][tmp.index[0]]), :]
        b7 = np.exp(np.log(tmp_b7['inc_rate']+1).sum()/7)-1
        tmp_a7 = tmp_.loc[np.logical_and(case_["date"] <= tmp['after_7'][tmp.index[0]],
                                     case_["date"] > tmp['date'][tmp.index[0]]), :]
        a7 = np.exp(np.log(tmp_a7['inc_rate']+1).sum()/7)-1
        tmp_b15 = tmp_.loc[np.logical_and(case_["date"] >= tmp['before_15'][tmp.index[0]],
                                     case_["date"] < tmp['date'][tmp.index[0]]), :]
        b15 = np.exp(np.log(tmp_b15['inc_rate']+1).sum()/15)-1
        tmp_a15 = tmp_.loc[np.logical_and(case_["date"] <= tmp['after_15'][tmp.index[0]],
                                     case_["date"] > tmp['date'][tmp.index[0]]), :]
        a15 = np.exp(np.log(tmp_a15['inc_rate']+1).sum()/15)-1
        res.append((s, p, b7, a7, b15, a15))

compare = pd.DataFrame(res, columns = ['state', 'policy', 'before_7', 'after_7', 'before_15', 'after_15']).dropna()
compare["diff_7"] = compare["after_7"] - compare["before_7"]
compare["diff_15"] = compare["after_15"] - compare["before_15"]

compare.to_csv("data/data/compare.csv")